In [1]:
!pip install django djangorestframework pyngrok


  Using cached djangorestframework-3.16.1-py3-none-any.whl.metadata (11 kB)
Using cached djangorestframework-3.16.1-py3-none-any.whl (1.1 MB)

   ---------------------------------------- 0/2 [pyngrok]
   ---------------------------------------- 0/2 [pyngrok]
   ---------------------------------------- 0/2 [pyngrok]
   ---------------------------------------- 0/2 [pyngrok]
   -------------------- ------------------- 1/2 [djangorestframework]
   -------------------- ------------------- 1/2 [djangorestframework]
   -------------------- ------------------- 1/2 [djangorestframework]
   -------------------- ------------------- 1/2 [djangorestframework]
   -------------------- ------------------- 1/2 [djangorestframework]
   -------------------- ------------------- 1/2 [djangorestframework]
   -------------------- ------------------- 1/2 [djangorestframework]
   -------------------- ------------------- 1/2 [djangorestframework]
   -------------------- ------------------- 1/2 [djangorestframew

In [2]:
!django-admin startproject userapi
%cd userapi


E:\jupiter\userapi


In [3]:
!python manage.py startapp users


In [4]:
with open("userapi/settings.py", "a") as f:
    f.write("\nINSTALLED_APPS += ['rest_framework', 'users']\n")


In [5]:
%%writefile users/models.py
from django.db import models

class User(models.Model):
    name = models.CharField(max_length=100)
    email = models.EmailField(unique=True)

    def __str__(self):
        return self.name


Overwriting users/models.py


In [6]:
!python manage.py makemigrations
!python manage.py migrate


Migrations for 'users':
  users\migrations\0001_initial.py
    + Create model User
Operations to perform:
  Apply all migrations: admin, auth, contenttypes, sessions, users
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying admin.0002_logentry_remove_auto_add... OK
  Applying admin.0003_logentry_add_action_flag_choices... OK
  Applying contenttypes.0002_remove_content_type_name... OK
  Applying auth.0002_alter_permission_name_max_length... OK
  Applying auth.0003_alter_user_email_max_length... OK
  Applying auth.0004_alter_user_username_opts... OK
  Applying auth.0005_alter_user_last_login_null... OK
  Applying auth.0006_require_contenttypes_0002... OK
  Applying auth.0007_alter_validators_add_error_messages... OK
  Applying auth.0008_alter_user_username_max_length... OK
  Applying auth.0009_alter_user_last_name_max_length... OK
  Applying auth.0010_alter_group_name_max_length... OK
  Applying

In [7]:
%%writefile users/serializers.py
from rest_framework import serializers
from .models import User

class UserSerializer(serializers.ModelSerializer):
    class Meta:
        model = User
        fields = '__all__'


Writing users/serializers.py


In [8]:
%%writefile users/views.py
from rest_framework.decorators import api_view
from rest_framework.response import Response
from rest_framework import status
from .models import User
from .serializers import UserSerializer

@api_view(['GET', 'POST'])
def user_list(request):
    if request.method == 'GET':
        users = User.objects.all()
        serializer = UserSerializer(users, many=True)
        return Response(serializer.data)
    elif request.method == 'POST':
        serializer = UserSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

@api_view(['GET', 'PUT', 'DELETE'])
def user_detail(request, pk):
    try:
        user = User.objects.get(pk=pk)
    except User.DoesNotExist:
        return Response({'error': 'User not found'}, status=status.HTTP_404_NOT_FOUND)

    if request.method == 'GET':
        serializer = UserSerializer(user)
        return Response(serializer.data)
    elif request.method == 'PUT':
        serializer = UserSerializer(user, data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)
    elif request.method == 'DELETE':
        user.delete()
        return Response({'message': 'User deleted successfully'}, status=status.HTTP_204_NO_CONTENT)


Overwriting users/views.py


In [9]:
%%writefile users/urls.py
from django.urls import path
from . import views

urlpatterns = [
    path('users/', views.user_list),
    path('users/<int:pk>/', views.user_detail),
]


Writing users/urls.py


In [10]:
%%writefile userapi/urls.py
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('api/', include('users.urls')),
]


Overwriting userapi/urls.py


In [11]:
from pyngrok import ngrok
public_url = ngrok.connect(8000)
print("Public URL:", public_url)


t=2025-10-24T03:39:13-0700 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
t=2025-10-24T03:39:13-0700 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
t=2025-10-24T03:39:13-0700 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [ ]:
!python manage.py runserver 0.0.0.0:8000
